In [1]:
import csv
import json
import time
import re

In [9]:
all_movies = []
with open('./data/disney_plus_titles.csv') as file:
    reader = csv.reader(file)
    headers = reader.__next__()

    # ['show_id', 'type', 'title', 'director', 'cast', 
    # 'country', 'date_added', 'release_year', 'rating', 
    # 'duration', 'listed_in', 'description']

    for row in reader:
        movies = dict(zip(headers, row))

        movies['show_id'] = movies.pop('show_id')
        movies['type'] = movies.pop('type')
        movies['title'] = movies.pop('title')

        movies['director'] = list(map(str.strip, movies.pop('director').split(',')))
        movies['cast'] = list(map(str.strip, movies.pop('cast').split(',')))
        movies['country'] = list(map(str.strip, movies.pop('country').split(',')))

        # CONVERTENDO EM ISODATE
        if movies['date_added']:
            time_struct = time.strptime(movies['date_added'].strip(), "%B %d, %Y")
            iso_date = time.strftime("%Y-%m-%dT%H:%M:%S.000Z", time_struct)
            movies['date_added'] = {'$date': iso_date}
        else:
            movies.pop('date_added', None)

        movies['release_year'] = int(movies.pop('release_year'))

        # pegando apenas caracteres numericos
        # lembrando que para type == movie a duration é por minutos
        # já type == tv-show a duration equivale ao número de temporadas (season)
        movies['duration'] = int(re.search( "\d." , movies.pop('duration')).group(0))


        movies['listed_in'] = list(map(str.strip, movies.pop('listed_in').split(',')))

        movies['rating'] = movies.pop('rating')
        movies['description'] = movies.pop('description')

        all_movies.append(movies)

    with open("./data/disney.json", "w") as file:
        json.dump(all_movies, file, indent=2)